# Desafio 3

## Importando

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd   


def ColabOuCode():
    '''
    Importa o DataFrame de acordo com a plataforma usada. Retorna um DataFrame.
    '''
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        df = pd.read_csv(
            '/content/drive/MyDrive/Colab Notebooks/TERA-desafio-3/data/fraud_detection_dataset.csv'
        )
    except:
        df = pd.read_csv('./data/fraud_detection_dataset.csv')
    
    return df


In [2]:
df = ColabOuCode()

Mounted at /content/drive


In [3]:
df.shape

(6362620, 11)

In [4]:
pd.set_option('display.max_columns', 11)
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [5]:
df.describe(percentiles=(0.9,0.95,0.99))

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06
mean,2.433972e+02,1.798619e+05,8.338831e+05,8.551137e+05,1.100702e+06,1.224996e+06,1.290820e-03,2.514687e-06
std,1.423320e+02,6.038582e+05,2.888243e+06,2.924049e+06,3.399180e+06,3.674129e+06,3.590480e-02,1.585775e-03
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.390000e+02,7.487194e+04,1.420800e+04,0.000000e+00,1.327057e+05,2.146614e+05,0.000000e+00,0.000000e+00
90%,3.990000e+02,3.654233e+05,1.822508e+06,1.970345e+06,2.914267e+06,3.194870e+06,0.000000e+00,0.000000e+00
95%,4.900000e+02,5.186342e+05,5.823702e+06,5.980262e+06,5.147230e+06,5.515716e+06,0.000000e+00,0.000000e+00
99%,6.810000e+02,1.615979e+06,1.602726e+07,1.617616e+07,1.237182e+07,1.313787e+07,0.000000e+00,0.000000e+00
max,7.430000e+02,9.244552e+07,5.958504e+07,4.958504e+07,3.560159e+08,3.561793e+08,1.000000e+00,1.000000e+00


In [7]:
df.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [28]:
for i in df.type.unique():
    newbal = df['newbalanceOrig'][df['type'] == i]
    oldbal = df['oldbalanceOrg'][df['type'] == i]
    dinMov = newbal - oldbal
    dinMov = np.abs(dinMov)
    dinMov = sum(dinMov)
    dinMov = np.round(dinMov, 2)

    print(i.ljust(10), dinMov)

PAYMENT    13724250333.7
TRANSFER   23529901460.21
CASH_OUT   63879853022.69
DEBIT      144418907.68
CASH_IN    236360342005.15
